## About iPython Notebooks ##

iPython Notebooks are interactive coding environments embedded in a webpage. You will be using iPython notebooks in this class. Make sure you fill in any place that says `# BEGIN CODE HERE #END CODE HERE`. After writing your code, you can run the cell by either pressing "SHIFT"+"ENTER" or by clicking on "Run" (denoted by a play symbol). Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All). 

 **What you need to remember:**

- Run your cells using SHIFT+ENTER (or "Run cell")
- Write code in the designated areas using Python 3 only
- Do not modify the code outside of the designated areas
- In some cases you will also need to explain the results. There will also be designated areas for that. 

Fill in your **NAME** and **AEM** below:

In [ ]:
NAME = "Konstantinos Oikonomou"
AEM = "3279"

---

# Assignment 3 - Ensemble Methods #

Welcome to your third assignment. This exercise will test your understanding on Ensemble Methods.

In [ ]:
# Always run this cell
import numpy as np
import pandas as pd

from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, GradientBoostingClassifier, VotingClassifier, StackingClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from statistics import mean

# USE THE FOLLOWING RANDOM STATE FOR YOUR CODE
RANDOM_STATE = 42

## Download the Dataset ##
Download the dataset using the following cell or from this [link](https://github.com/sakrifor/public/tree/master/machine_learning_course/EnsembleDataset) and put the files in the same folder as the .ipynb file. 
In this assignment you are going to work with a dataset originated from the [ImageCLEFmed: The Medical Task 2016](https://www.imageclef.org/2016/medical) and the **Compound figure detection** subtask. The goal of this subtask is to identify whether a figure is a compound figure (one image consists of more than one figure) or not. The train dataset consits of 4197 examples/figures and each figure has 4096 features which were extracted using a deep neural network. The *CLASS* column represents the class of each example where 1 is a compoung figure and 0 is not. 


In [ ]:
import urllib.request
url_train = 'https://github.com/sakrifor/public/raw/master/machine_learning_course/EnsembleDataset/train_set.csv'
filename_train = 'train_set.csv'
urllib.request.urlretrieve(url_train, filename_train)
url_test = 'https://github.com/sakrifor/public/raw/master/machine_learning_course/EnsembleDataset/test_set_noclass.csv'
filename_test = 'test_set_noclass.csv'
urllib.request.urlretrieve(url_test, filename_test)

('test_set_noclass.csv', <http.client.HTTPMessage at 0x7eff512b80d0>)

In [ ]:
# Run this cell to load the data
train_set = pd.read_csv("train_set.csv").sample(frac=1).reset_index(drop=True)
train_set.head()
X = train_set.drop(columns=['CLASS'])
y = train_set['CLASS'].values

## 1.0 Testing different ensemble methods ##
In this part of the assignment you are asked to create and test different ensemble methods using the train_set.csv dataset. You should use **10-fold cross validation** for your tests and report the average f-measure and accuracy of your models.

### !!! Use n_jobs=-1 where is posibble to use all the cores of a machine for running your tests ###

### 1.1 Voting ###
Create a voting classifier which uses three estimators/classifiers. Test both soft and hard voting and choose the best one.

In [ ]:
# BEGIN CODE HERE

cls1 = AdaBoostClassifier(n_estimators=10, random_state=RANDOM_STATE) # Classifier #1 
cls2 = RandomForestClassifier(n_estimators=10, random_state=RANDOM_STATE) # Classifier #2 
cls3 = GradientBoostingClassifier(n_estimators=10, learning_rate=1.0, max_depth=5, random_state=RANDOM_STATE) # Classifier #1
vcls = VotingClassifier(estimators=[('ab', cls1), ('rf', cls2), ('gb', cls3)], voting='soft') # Voting Classifier

#Hard: F-Score 81.86%, Accuracy 78.19% 
#Sort: F-Score 82.13%, Accuracy 78.31% 

cls1.fit(X, y)
cls2.fit(X, y)
cls3.fit(X, y)
vcls.fit(X, y)

cv = KFold(n_splits=10, random_state=RANDOM_STATE, shuffle=True)

fmeasure = cross_val_score(vcls, X, y, scoring='f1', cv=cv, n_jobs=-1)
avg_fmeasure = mean(fmeasure) # The average f-measure

accuracy = cross_val_score(vcls, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
avg_accuracy = mean(accuracy) # The average accuracy 

#END CODE HERE

In [ ]:
print("Classifier:")
print(vcls)
print("F1-Score:{} & Accuracy:{}".format(avg_fmeasure,avg_accuracy))

Classifier:
VotingClassifier(estimators=[('ab',
                              AdaBoostClassifier(algorithm='SAMME.R',
                                                 base_estimator=None,
                                                 learning_rate=1.0,
                                                 n_estimators=10,
                                                 random_state=42)),
                             ('rf',
                              RandomForestClassifier(bootstrap=True,
                                                     ccp_alpha=0.0,
                                                     class_weight=None,
                                                     criterion='gini',
                                                     max_depth=None,
                                                     max_features='auto',
                                                     max_leaf_nodes=None,
                                                     max_samples=None,
      

### 1.2 Stacking ###
Create a stacking classifier which uses two estimators/classifiers. Try different classifiers for the combination of the initial classifiers. Report your results in the following cell.

In [ ]:
# BEGIN CODE HERE

#cls1 = AdaBoostClassifier(n_estimators=10, random_state=RANDOM_STATE)
cls1 = RandomForestClassifier(n_estimators=10, random_state=RANDOM_STATE) # Classifier #1 
cls2 = GradientBoostingClassifier(n_estimators=10, learning_rate=1.0, max_depth=5, random_state=RANDOM_STATE) # Classifier #2 
scls = StackingClassifier(estimators=[('rf', cls1), ('gb', cls2)]) # Stacking Classifier

cls1.fit(X, y)
cls2.fit(X, y)
scls.fit(X, y)

cv = KFold(n_splits=10, random_state=RANDOM_STATE, shuffle=True)

fmeasure = cross_val_score(scls, X, y, scoring='f1', cv=cv, n_jobs=-1)
avg_fmeasure = mean(fmeasure)

accuracy = cross_val_score(scls, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
avg_accuracy = mean(accuracy)

#END CODE HERE

In [ ]:
print("Classifier:")
print(scls)
print("F1-Score:{} & Accuracy:{}".format(avg_fmeasure,avg_accuracy))

Classifier:
StackingClassifier(cv=None,
                   estimators=[('ab',
                                AdaBoostClassifier(algorithm='SAMME.R',
                                                   base_estimator=None,
                                                   learning_rate=1.0,
                                                   n_estimators=10,
                                                   random_state=42)),
                               ('gb',
                                RandomForestClassifier(bootstrap=True,
                                                       ccp_alpha=0.0,
                                                       class_weight=None,
                                                       criterion='gini',
                                                       max_depth=None,
                                                       max_features='auto',
                                                       max_leaf_nodes=None,
                     

### 1.3 Report the results ###  
Report the results of your experiments in the following cell. How did you choose your initial classifiers? 

AdaBoost & Random Forest: F1 Score 81.84%, Accuracy 77.69%, fastest way and decent results

AdaBoost & GradientBoosting: F1 Score 80.60%, Accuracy 76.62%, slower and worse results

RandomForest & GradientBoosting: F1 Score 83.01%, Accuracy 79.50%, slower but has best results

## 2.0 Randomization ##

**2.1** You are asked to create three ensembles of decision trees where each one uses a different method for producing homogeneous ensembles. Compare them with a simple decision tree classifier and report your results in the dictionaries (dict) below using as key the given name of your classifier and as value the f1/accuracy score. The dictionaries should contain four different elements.  

In [ ]:
# BEGIN CODE HERE
ens1 = DecisionTreeClassifier(criterion='entropy', splitter='best', random_state=RANDOM_STATE)
ens2 = DecisionTreeClassifier(criterion='gini', splitter='random', random_state=RANDOM_STATE)
ens3 = DecisionTreeClassifier(criterion='entropy', splitter='random', random_state=RANDOM_STATE)
tree = DecisionTreeClassifier(random_state=RANDOM_STATE)

ens1.fit(X, y)
ens2.fit(X, y)
ens3.fit(X, y)
tree.fit(X, y)

cv = KFold(n_splits=10, random_state=RANDOM_STATE, shuffle=True)

#f1-scores
fens1 = cross_val_score(ens1, X, y, scoring='f1', cv=cv, n_jobs=-1)
fens1 = mean(fens1)

fens2 = cross_val_score(ens2, X, y, scoring='f1', cv=cv, n_jobs=-1)
fens2 = mean(fens2)

fens3 = cross_val_score(ens3, X, y, scoring='f1', cv=cv, n_jobs=-1)
fens3 = mean(fens3)

ftree = cross_val_score(tree, X, y, scoring='f1', cv=cv, n_jobs=-1)
ftree = mean(ftree)

f_measures = {'Ensemble with igain':fens1, 'Ensemble with ramdom split':fens2, 'Ensemble with igain and random split':fens3, 'Simple Decision':ftree}

#accuracies
aens1 = cross_val_score(ens1, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
aens1 = mean(aens1)

aens2 = cross_val_score(ens2, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
aens2 = mean(aens2)

aens3 = cross_val_score(ens3, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
aens3 = mean(aens3)

atree = cross_val_score(tree, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
atree = mean(atree)

accuracies = {'Ensemble with igain':aens1, 'Ensemble with ramdom split':aens2, 'Ensemble with igain and random split':aens3, 'Simple Decision':atree}
#END CODE HERE

In [ ]:
print(ens1)
print(ens2)
print(ens3)
print(tree)
for name,score in f_measures.items():
    print("Classifier:{} -  F1:{}".format(name,score))
for name,score in accuracies.items():
    print("Classifier:{} -  Accuracy:{}".format(name,score))

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=42, splitter='best')
DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=42, splitter='random')
DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=None, max_features=None, max_leaf_

**2.2** Describe your classifiers and your results.

Information Gain and Best Split Decision Tree: F1 Score 76.07%, Accuracy 71.59%

Gini and Random Split Decision Tree: F1 Score 75.20%, Accuracy 70.97%

Information Gain and Radnom Split Decision Tree: F1 Score 74.76%, Accuracy 70.49%

Gini and Best Split Decision Tree: F1 Score 74.81%, Accuracy 70.44%

The Best Classifier appears to be the igain with best split decision tree, outscoring every other tree classifier

**2.3** Increasing the number of estimators in a bagging classifier can drastically increase the training time of a classifier. Is there any solution to this problem? Can the same solution be applied to boosting classifiers?

YOUR ANSWER HERE

## 3.0 Creating the best classifier ##

**3.1** In this part of the assignment you are asked to train the best possible ensemble! Describe the process you followed to achieve this result. How did you choose your classifier and your parameters and why. Report the f-measure & accuracy (10-fold cross validation) of your final classifier and results of classifiers you tried in the cell following the code. Can you achieve an accuracy over 83-84%?

In [ ]:
# BEGIN CODE HERE

cls1 = KNeighborsClassifier()
#cls2 = DecisionTreeClassifier(criterion='entropy', splitter='best', random_state=RANDOM_STATE)
cls2 = RandomForestClassifier(n_estimators=10, random_state=RANDOM_STATE) 
cls3 = GradientBoostingClassifier(n_estimators=10, learning_rate=0.1, random_state=RANDOM_STATE) 
#cls4 = GaussianNB()

best_cls = StackingClassifier([('knn',cls1), ('dt',cls2), ('rf',cls3)])

cls1.fit(X, y)
cls2.fit(X, y)
cls3.fit(X, y)
#cls4.fit(X, y)
best_cls.fit(X, y)

cv = KFold(n_splits=10, random_state=RANDOM_STATE, shuffle=True)

fbest = cross_val_score(best_cls, X, y, scoring='f1', cv=cv, n_jobs=-1)
best_fmeasure = mean(fbest)

abest = cross_val_score(best_cls, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
best_accuracy = mean(abest)

#END CODE HERE

In [ ]:
print("Classifier:")
print(best_cls)
print("F1-Score:{} & Accuracy:{}".format(best_fmeasure,best_accuracy))

Classifier:
StackingClassifier(cv=None,
                   estimators=[('knn',
                                KNeighborsClassifier(algorithm='auto',
                                                     leaf_size=30,
                                                     metric='minkowski',
                                                     metric_params=None,
                                                     n_jobs=None, n_neighbors=5,
                                                     p=2, weights='uniform')),
                               ('dt',
                                RandomForestClassifier(bootstrap=True,
                                                       ccp_alpha=0.0,
                                                       class_weight=None,
                                                       criterion='gini',
                                                       max_depth=None,
                                                       max_features='auto',
    

**3.2** Describe the process you followed to achieve this result. How did you choose your classifier and your parameters and why. Report the f-measure & accuracy (10-fold cross validation) of your final classifier and results of classifiers you tried in the cell following the code.

1st Measure: knn, randomForest, GradientBoosting: F1-Score: 84.84%, Accuracy: 81.64%

2nd Measure: knn, tree, randomForest, GradientBoosting: F1-Score: 84.69% Accuracy: 81.45%

3rd Measure: knn, randomForest, GradientBoosting, gaussianNB: F1-Score: 84.77%, Accuracy: 81.55%

All the classifiers have similar results but the one with KNearestNeighbors, RandomForest and GradientBoosting has the heighest score so far


**3.3** Create a classifier that is going to be used in production - in a live system. Use the *test_set_noclass.csv* to make predictions. Store the predictions in a list.  

In [ ]:
# BEGIN CODE HERE

cls1 = KNeighborsClassifier()
cls2 = RandomForestClassifier(n_estimators=10, random_state=RANDOM_STATE) 
cls3 = GradientBoostingClassifier(n_estimators=10, learning_rate=0.1, random_state=RANDOM_STATE) 

cls = StackingClassifier([('knn',cls1), ('dt',cls2), ('rf',cls3)])

cls1.fit(X, y)
cls2.fit(X, y)
cls3.fit(X, y)
cls.fit(X, y)

X_test = pd.read_csv("test_set_noclass.csv").sample(frac=1).reset_index(drop=True)

predictions = cls.predict(X_test)

#END CODE HERE

In [ ]:
print(cls)
print(predictions)

StackingClassifier(cv=None,
                   estimators=[('knn',
                                KNeighborsClassifier(algorithm='auto',
                                                     leaf_size=30,
                                                     metric='minkowski',
                                                     metric_params=None,
                                                     n_jobs=None, n_neighbors=5,
                                                     p=2, weights='uniform')),
                               ('dt',
                                RandomForestClassifier(bootstrap=True,
                                                       ccp_alpha=0.0,
                                                       class_weight=None,
                                                       criterion='gini',
                                                       max_depth=None,
                                                       max_features='auto',
                

LEAVE HERE ANY COMMENTS ABOUT YOUR CLASSIFIER

#### This following cell will not be executed. The test_set.csv with the classes will be made available after the deadline and this cell is for testing purposes!!! Do not modify it! ###

In [ ]:
from sklearn.metrics import f1_score,accuracy_score
final_test_set = pd.read_csv('test_set.csv')
ground_truth = final_test_set['CLASS']
print("Accuracy:{}".format(accuracy_score(predictions,ground_truth)))
print("F1-Score:{}".format(f1_score(predictions,ground_truth)))